In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.src.layers import BatchNormalization

from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models, Input, Model, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tcn import TCN
from sklearn.metrics import r2_score, mean_absolute_error


##############################################
# Définition des métriques personnalisées
##############################################
def r2_metric(y_true, y_pred):
    """Calcul du coefficient de détermination R²."""
    ss_res = tf.reduce_sum(tf.square(y_true - y_pred))
    ss_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    return 1 - ss_res / ss_tot


def accuracy_5_metric(y_true, y_pred):
    """
    Calcul de l'accuracy à 5% :
    Renvoie la proportion de prédictions dont l'erreur relative est <= 5%.
    """
    # Erreur relative : |y_true - y_pred| / (|y_true| + epsilon)
    error = tf.abs((y_true - y_pred) / (tf.abs(y_true)))
    correct = tf.cast(error <= 0.05, tf.float32)

    return tf.reduce_mean(correct)


##############################################
# 1) Création des séquences
##############################################
def create_sequences(values, sequence_length=60):
    """
    Crée des séquences glissantes de longueur 'sequence_length'.
    'values' doit être un tableau 2D de forme (nombre_samples, nombre_caractéristiques).
    Renvoie X et y avec :
       X: (nombre_samples - sequence_length, sequence_length, nombre_caractéristiques)
       y: (nombre_samples - sequence_length, 1)
    """
    X, y = [], []
    for i in range(sequence_length, len(values)):
        X_window = values[i - sequence_length: i]
        y_value = values[i]  # Prédire la valeur suivante
        X.append(X_window)
        y.append(y_value)
    X = np.array(X)
    y = np.array(y).reshape(-1, 1)
    return X, y


##############################################
# 2) Fonction build_model utilisant TCN (de tcn.py) avec empilement correct
##############################################
def build_model(input_shape, learning_rate=0.0005):
    """
    Construit un modèle de régression utilisant 2 couches TCN empilées.
    La première couche TCN renvoie toute la séquence (return_sequences=True),
    et la deuxième couche TCN renvoie uniquement la dernière sortie (return_sequences=False).
    Une couche Dropout est ajoutée entre les deux.
    Enfin, une Dense(1) produit la prédiction.
    Les métriques utilisées sont MAE, R² et l'accuracy à 5%.
    """
    model = models.Sequential([
        layers.Input(shape=input_shape),
        # Première couche TCN renvoie toute la séquence
        TCN(
            nb_filters=2,
            kernel_size=3,
            nb_stacks=1,
            dilations=[1, 4, 16, 64, 1024],
            padding='causal',
            dropout_rate=0.2,
            #return_sequences=True
        ),
        layers.Dense(1)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='mean_squared_error',
        metrics=['mean_absolute_error', r2_metric, accuracy_5_metric]
    )
    return model


##############################################
# 3) Workflow principal
##############################################
def main():
    # (A) Charger le CSV
    df = pd.read_csv("/Users/welto/Library/CloudStorage/OneDrive-CentraleSupelec/3A/DL/Project/NASDAQ_100.csv")
    # On utilise la colonne 'close' pour la prévision (attention à la casse)
    close_prices = df['close'].values.reshape(-1, 1)

    # (B) Mise à l'échelle avec StandardScaler
    scaler = StandardScaler()
    scaled_prices = scaler.fit_transform(close_prices)

    # (C) Création des séquences (par exemple, 60 jours d'entrée)
    sequence_length = 60
    X, y = create_sequences(scaled_prices, sequence_length=sequence_length)
    print("Forme de X :", X.shape, "Forme de y :", y.shape)

    # (D) Découpage en ensembles Train/Val/Test (70/15/15 %)
    train_size = int(len(X) * 0.7)
    val_size = int(len(X) * 0.15)
    X_train, y_train = X[:train_size], y[:train_size]
    X_val, y_val = X[train_size:train_size + val_size], y[train_size:train_size + val_size]
    X_test, y_test = X[train_size + val_size:], y[train_size + val_size:]
    print("Train :", X_train.shape, y_train.shape)
    print("Val   :", X_val.shape, y_val.shape)
    print("Test  :", X_test.shape, y_test.shape)

    # (E) Construction du modèle TCN
    model = build_model(input_shape=(sequence_length, 1), learning_rate=0.0005)
    model.summary()

    # (F) Entraînement du modèle
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_data=(X_val, y_val),
        callbacks=[
            #early_stopping,
            reduce_lr
        ]
    )

    # Tracer l'évolution de la perte
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['loss'], label='Perte entraînement')
    plt.plot(history.history['val_loss'], label='Perte validation')
    plt.xlabel('Epoch')
    plt.ylabel('MSE')
    plt.yscale("log")
    plt.title("Évolution de la perte")
    plt.legend()
    plt.show()

    # (G) Évaluation et prédiction
    test_loss = model.evaluate(X_test, y_test)
    print("Test Loss :", test_loss)
    y_pred_scaled = model.predict(X_test)
    y_test_unscaled = scaler.inverse_transform(y_test)
    y_pred_unscaled = scaler.inverse_transform(y_pred_scaled)

    # (H) Tracer les prédictions vs valeurs réelles pour les 100 premiers échantillons de test
    n_plot = 100
    plt.figure(figsize=(10, 5))
    plt.plot(y_test_unscaled[:n_plot], label='Prix réel')
    plt.plot(y_pred_unscaled[:n_plot], label='Prix prédit')
    plt.xlabel("Index d'échantillon de test")
    plt.ylabel("Prix de clôture")
    plt.title("Prédictions TCN vs. Prix réels (100 premiers échantillons)")
    plt.legend()
    plt.show()


if __name__ == "__main__":
    main()

Forme de X : (9656, 60, 1) Forme de y : (9656, 1)
Train : (6759, 60, 1) (6759, 1)
Val   : (1448, 60, 1) (1448, 1)
Test  : (1449, 60, 1) (1449, 1)


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ tcn_10 (TCN)                    │ (None, 2)              │           138 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 141 (564.00 B)

 Trainable params: 141 (564.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
212/212 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy_5_metric: 0.0521 - loss: 0.5820 - mean_absolute_error: 0.5384 - r2_metric: -12.7909 - val_accuracy_5_metric: 0.0000e+00 - val_loss: 0.6338 - val_mean_absolute_error: 0.6174 - val_r2_metric: -1499.1277 - learning_rate: 5.0000e-04
Epoch 2/100
212/212 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy_5_metric: 0.0245 - loss: 0.1251 - mean_absolute_error: 0.2515 - r2_metric: -1.8051 - val_accuracy_5_metric: 0.0095 - val_loss: 0.6406 - val_mean_absolute_error: 0.6293 - val_r2_metric: -1558.1284 - learning_rate: 5.0000e-04
Epoch 3/100
212/212 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy_5_metric: 0.4202 - loss: 0.0910 - mean_absolute_error: 0.1885 - r2_metric: -0.9447 - val_accuracy_5_metric: 0.0088 - val_loss: 0.6674 - val_mean_absolute_error: 0.6594 - val_r2_metric: -1670.4895 - learning_rate: 5.0000e-04
Epoch 4/100
212/212 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy_5_metric: 0.3641 - loss: 0.0705 - mean_absolute_error: 0.1629 - r2_

KeyboardInterrupt: 